## Notebook to produce plots for the Gennaro & Khandrika ISR on up-the-ramp fitting

In [ ]:
import pandas as pd
import numpy as np
from ramp_utils.ramp import RampTimeSeq,RampMeasurement
from ramp_utils.fitter import IterativeFitter
import matplotlib.pyplot as plt
import time, sys, pickle, bz2
from multiprocessing import Pool

%matplotlib notebook

#### Create a WFC3 ramp

In [ ]:
myramp = RampTimeSeq('HST/WFC3/IR',15,samp_seq='SPARS100')


#### Choose the "true" flux (in e/s), and specify a desired sequence of Cosmic Rays impacts

In [ ]:
myflux = 1.

#CRdict = {'times':[620., 80.,400.,980],'counts':[300,500,250,290]}
CRdict = {'times':[640.],'counts':[100]}
#CRdict = None  # For no cosmic rays

#### Create a measurement.   

In [ ]:
mymeas = RampMeasurement(myramp,myflux,CRdict=CRdict)  #an example for WFC3 ramps

#### Plot showing the CR rejection method in calwf3

In [ ]:
plt.style.use('bmh')
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 20
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['figure.titlesize'] = 18


f,ax = plt.subplots(2,2,figsize=(12,6),sharex='col')


ax[0,0].scatter(mymeas.RTS.read_times[mymeas.RTS.kept_reads],mymeas.noisy_counts_reads[mymeas.RTS.kept_reads],label='Measured Counts',s=50)

ax[0,0].set_ylim(10999,11800)
ydw,yup = ax[0,0].get_ylim()

if mymeas.CRdict is not None:
    cmax = 0.
    for t,c in zip(mymeas.CRdict['times'],mymeas.CRdict['counts']):
        ax[0,0].plot([t,t],[ydw+50,ydw+50+c],label='CR hit',c='#555555')
        if c > cmax:
            cmax = c

dt = mymeas.RTS.group_times[1:]-mymeas.RTS.group_times[:-1]
    
linslope = (mymeas.noisy_counts[-1]-mymeas.noisy_counts[0])/(mymeas.RTS.group_times[-1]-mymeas.RTS.group_times[0])
linfit = mymeas.noisy_counts[0] + linslope*(mymeas.RTS.group_times-mymeas.RTS.group_times[0])

ax[0,0].plot(mymeas.RTS.group_times,linfit,c='orange',label='Initial guess')
ax[0,0].legend()

ax[1,0].scatter(mymeas.RTS.group_times,linfit-mymeas.noisy_counts,label='Residuals',s=50)

if mymeas.CRdict is not None:
    cmax = 0.
    for t,c in zip(mymeas.CRdict['times'],mymeas.CRdict['counts']):
        ax[1,0].plot([t,t],[0.5*c,-0.5*c],label='CR hit',c='#555555')
        if c > cmax:
            cmax = c

            
ind_rates = (mymeas.noisy_counts[1:]-mymeas.noisy_counts[:-1])/dt
            
ax[0,1].scatter(0.5*(mymeas.RTS.read_times[mymeas.RTS.kept_reads][1:]+mymeas.RTS.read_times[mymeas.RTS.kept_reads][:-1]),
                ind_rates,
                label='Measured Countrate',s=50)

ydw,yup = ax[0,1].get_ylim()

if mymeas.CRdict is not None:
    cmax = 0.
    for t,c in zip(mymeas.CRdict['times'],mymeas.CRdict['counts']):
        ax[0,1].plot([t,t],[ydw,ydw+c/100],label='CR hit',c='#555555')
        if c > cmax:
            cmax = c


initial_guess = np.mean(ind_rates)
ax[0,1].plot(mymeas.RTS.group_times,np.ones_like(mymeas.RTS.group_times)*initial_guess,c='orange',label='Initial guess')
 
    
var_signal_per_diff = dt * initial_guess * mymeas.gain
stddev = np.sqrt(var_signal_per_diff+2.*np.square(mymeas.RON_e))

#ax[1,1].scatter(0.5*(mymeas.RTS.group_times[1:]+mymeas.RTS.group_times[:-1]),
#                ind_rates-initial_guess,
#                label='Residuals',s=50)
ind_electrons = (mymeas.noisy_counts[1:]-mymeas.noisy_counts[:-1])*mymeas.gain
electrons_guess = initial_guess*mymeas.gain*dt

ax[1,1].scatter(0.5*(mymeas.RTS.group_times[1:]+mymeas.RTS.group_times[:-1]),
               (ind_electrons-electrons_guess)/stddev,label='Residuals/$\sigma$')
ax[1,1].axhline(4,color='orange')
ax[1,1].axhline(-4,color='orange')
ax[1,1].fill_between([-20,1290], [4,4], [-4,-4],color='orange',alpha=0.25)
             
#ax[1,1].errorbar(0.5*(mymeas.RTS.group_times[1:]+mymeas.RTS.group_times[:-1]),
#                ,
#                 yerr=4*stddev,
#                label='Residuals',fmt='o')

ax[1,1].set_xlim(-20,1290)

for axxx in ax:
    for axx in axxx:
        axx.legend()
        axx.set_facecolor('#FFFFFF')
        axx.set_facecolor('#FFFFFF')

ax[1,0].set_xlabel('Time [s]')
ax[0,0].set_ylabel('Counts')
ax[1,0].set_ylabel('Counts')
ax[0,1].set_ylabel(r'Countrate [s$^{-1}$]')
ax[1,1].set_ylabel(r'Counts')

nsp1  = '   '    
nsp2  = '   '    
    
plt.figtext(0.29,0.9625,nsp1+"calwf3"+nsp1, va="center", ha="center", size=18,bbox={'facecolor':'gray', 'edgecolor':'black','alpha':.1, 'pad':4})
plt.figtext(0.78,0.9625,nsp2+"This work"+nsp2, va="center", ha="center", size=18,bbox={'facecolor':'gray','edgecolor':'black', 'alpha':.1, 'pad':4})

plt.tight_layout()
plt.subplots_adjust(wspace=.25,hspace=.25,left=.1,top=0.925)


f.savefig('/user/gennaro/Functional_work/ISRs/Up_the_ramp_fitting/Figs/calwf3_CR_jump.pdf')

In [ ]:
stddev
